# example code for training model & data taken from here

https://matheusfacure.github.io/python-causality-handbook/21-Meta-Learners.html

# imports

In [2]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from dsf_2023.dataloader import load_data
import importlib.resources
from functools import reduce

# load data

In [3]:
data = load_data()

In [4]:
data.keys()

dict_keys(['invest_email_rnd_df', 'invest_email_biased_df', 'invest_email_df'])

In [5]:
test_df = data['invest_email_rnd_df']
train_df = data['invest_email_biased_df']
train_df.head()

,age,income,insurance,invested,em1,em2,em3,converted
0,44.1,5483.80,6155.29,14294.81,0,0,1,0
1,39.8,2737.92,50069.40,7468.15,1,0,0,0
2,49.0,2712.51,5707.08,5095.65,0,0,1,1
3,39.7,2326.37,15657.97,6345.20,0,0,0,0
4,35.3,2787.26,27074.44,14114.86,1,1,0,0


In [6]:
y = "converted"
T = "em1"
X = ["age", "income", "insurance", "invested"]

# fit S-learner

As per book however we fit a RandomForestRegressor rather than LGBMRegressor

In [7]:

np.random.seed(123)
s_learner = RandomForestRegressor(
    max_depth=5, min_samples_leaf=10, 
)
s_learner.fit(train_df[X+[T]], train_df[y]);

# get preds

## s-learner preds

In [8]:
def get_preds(df: pd.DataFrame, model: RandomForestRegressor) -> pd.DataFrame:
    preds_df = pd.DataFrame(dict(
        control_pred=model.predict(df[X].assign(**{T: 0})),
        treated_pred=model.predict(df[X].assign(**{T: 1})),
        w=df[T].to_numpy(),
        y=df[y].to_numpy(),
    )).assign(
        cate=lambda x: x['treated_pred'] - x['control_pred'],
        cate_imputed=lambda x: np.where(x['w'] == 1, x['y'] - x['control_pred'], x['treated_pred'] - x['y']),
        marginal=lambda x: np.where(x['w'] == 1, x['treated_pred'], x['control_pred'])
    )

    return preds_df


In [9]:
train_preds_df = get_preds(df=train_df, model=s_learner)
test_preds_df = get_preds(df=test_df, model=s_learner)

## add random model preds

helpful for when comparing performance of random model vs. actual s-learner

In [10]:
all_preds = {}
for df in [train_preds_df, test_preds_df]:
    loc = df['y'].mean()
    scale = df['y'].var() ** 0.5
    for idx in range(1, 11):
        # add 10 random model predictions
        df[f'RANDOM_{idx}'] = np.random.normal(
            size=df.shape[0], loc=loc, scale=scale
        )

# Causal ML Metrics

## lift

### helper functions

In [11]:
def add_bins(
    df: pd.DataFrame,
    outcome_col: str,
    treated_col: str,
    cate_col: str,
    bins: int =10
) -> pd.DataFrame:
    """
    creates bins using qcut with pre-processing done before hand to ensure
    that the number of bins specified will always be created.
    
    qcut can fail to do this when for example a left value of bin threshold
    that divides one bin from the next is repeated (i.e. threshold.left is 
    the same for two bins), which can happen but in subsequent bins the thresholds
    can then change. We achieve this by creating a unique rank column first
    based on cate before using qcut on this. 
    """
    err_msg = f'err you have too many bins vs n_obs: bins={bins} n_obs={df.shape[0]:0,.0f}'
    assert df.shape[0] > bins, err_msg


    df = pd.DataFrame(dict(
            y = df[outcome_col].to_numpy(),
            w = df[treated_col].to_numpy(),
            cate=df[cate_col].to_numpy(),                        
    )).sort_values(
      ['cate'], ascending=False
    ).reset_index(
      drop=True
    ).assign(
      rank=lambda x: x.index,
      bin_sort_id=lambda x: pd.qcut(x['rank'], q=bins, labels=False),
    ).assign(
      bin_min=lambda x: x.groupby(['bin_sort_id'])['cate'].transform('min'),
      bin_max=lambda x: x.groupby(['bin_sort_id'])['cate'].transform('max'),     
    )
    df['bin'] = df.apply(lambda x: f'{x.bin_min : 0.2f}-{x.bin_max: 0.2f}', axis=1)

    return df


def get_lift_for_bin(
    df: pd.DataFrame,
    outcome_col: str,
    treated_col: str,
    cate_col: str,
) -> pd.DataFrame:
    """
    each bin will have n_treated customers (bin_n_tr) with y_treated values (bin_y_tr).
    Similarly we will have this for controls (bin_n_ct and bin_y_ct). We also will have
    predited cate for treated and control (bin_cate_tr & bin_cate_ct)
    
    We estimate the lift of a bin using the following definition:
      lift = (x['bin_y_tr']/x['bin_n_tr']) - (x['bin_y_ct']/x['bin_n_ct'])
    while predicted lift is defined as:
      pred_cate = (x['bin_cate_tr'] + x['bin_cate_ct']) / (x['bin_n_tr']+ x['bin_n_ct'])

    outputs a data frame with a single row with the following fields:
       a dataframe with fields added to it
        n_tr: # of treated customers assigned to a bin
        n_ct: # of control customers assigned to a bin
        y_tr: avg. outcome value for treated assigned to a bin
        y_ct: avg. outcome value for control assigned to a bin
        lift: (y_tr -  y_ct) assigned to a bin
        cate_tr: predicted conditional avg. treatment effect for treated group assigned to a bin
        cate_ct: predicted conditional avg. treatment effect for control group assigned to a bin   
        pred_cate: predicted conditional avg. treatment effect (treated + control group) assigned to a bin


    this function is flexible to be used if wished at a bin level
    or cumulative bin level as the method will aggregate n, y and predicted
    cate for treated and controls before estimating avg., lift and predicted cate
    """

    df = df.copy()

    df = pd.DataFrame(dict(
        n_tr=[df[treated_col].sum()],
        n_ct=[(1-df[treated_col]).sum()],

        y_tr=[(df[outcome_col] * df[treated_col]).sum()],
        y_ct=[(df[outcome_col] * (1 - df[treated_col])).sum()],

        cate_tr=[(df[cate_col] * df[treated_col]).sum()],
        cate_ct=[(df[cate_col] * (1 - df[treated_col])).sum()],        
    )).assign(
        y_tr = lambda x: x['y_tr']/x['n_tr'],
        y_ct = lambda x: x['y_ct']/x['n_ct'],
        lift = lambda x: x['y_tr'] -  x['y_ct'],
        #pred_cate_tr= lambda x: x['cate_tr'] / x['n_tr'],
        #pred_cate_ct= lambda x: x['cate_ct'] / x['n_ct'],
        pred_cate = lambda x: (x['cate_tr'] + x['cate_ct']) / (x['n_tr']+ x['n_ct'])
    )
    
    return df


def get_lift_effects_df(
    df: pd.DataFrame,
    outcome_col: str,
    treated_col: str,
    cate_col: str,
    bins: int
) -> pd.DataFrame:
    """
    each bin will have n_treated customers (bin_n_tr) with y_treated values (bin_y_tr).
    Similarly we will have this for controls (bin_n_ct and bin_y_ct). We also will have
    predited cate for treated and control (bin_cate_tr & bin_cate_ct)

    for each bin we estimate the observed uplift:
        (x['bin_y_tr']/x['bin_n_tr']) - (x['bin_y_ct']/x['bin_n_ct'])
    vs predicted uplift uplift:
        (x['bin_cate_tr'] + x['bin_cate_ct']) / (x['bin_n_tr']+ x['bin_n_ct'])

    outputs a data frame with n_row equal to the number of bins specified with fields:
        bin_sort_id: bin id
        bin: the left and right thresholds based on predicted cate that define the bin
        n_tr: # of treated customers assigned to a bin
        n_ct: # of control customers assigned to a bin
        y_tr: avg. outcome value for treated assigned to a bin
        y_ct: avg. outcome value for control assigned to a bin
        lift: (y_tr -  y_ct) assigned to a bin
        cate_tr: predicted conditional avg. treatment effect for treated group assigned to a bin
        cate_ct: predicted conditional avg. treatment effect for control group assigned to a bin   
        pred_cate: predicted conditional avg. treatment effect (treated + control group) assigned to a bin
        global_lift:
            y_global_tr: (x['y_tr'] * x['n_tr']).sum() / x['n_tr'].sum()
            y_global_ct: (x['y_ct'] * x['n_ct']).sum() / x['n_ct'].sum()
            global_lift: (y_global_tr - y_global_ct)
    """

    assert set(df[treated_col].unique()) == {0, 1}

    df_binned = add_bins(
        df=df,
        outcome_col=outcome_col,
        treated_col=treated_col,
        cate_col=cate_col,
        bins=bins
    )

    all_bin_uplifts = []
    for bin_info, bin_df in df_binned.groupby(['bin_sort_id', 'bin']):
        bin_sort_id, bin = bin_info
        all_bin_uplifts.append(get_lift_for_bin(
            df=bin_df,
            outcome_col='y',
            treated_col='w',
            cate_col='cate'
        ).assign(
            bin_sort_id = bin_sort_id, 
            bin = bin, 
        ))

    all_bin_uplifts = pd.concat( all_bin_uplifts, axis=0).sort_values(
        ['bin_sort_id'], ascending=True
    ).reset_index(drop=True)


    y_tr = (all_bin_uplifts['y_tr']*all_bin_uplifts['n_tr']).sum()/all_bin_uplifts['n_tr'].sum()
    y_ct = (all_bin_uplifts['y_ct']*all_bin_uplifts['n_ct']).sum()/all_bin_uplifts['n_ct'].sum()
    all_bin_uplifts['global_lift'] = (y_tr - y_ct)

    return all_bin_uplifts 


def kendall_rank_correlation(df: pd.DataFrame, lift_col='lift', pred_lift_col='pred_cate'):
    """
    expected input must be from running the below function first:
    bin_sort_id + lift_col + pred_lift_col (i.e. pred_lift)! 
    the following function can be ran in preparation for it

    df = get_lift_effects_df(
    df,
    outcome_col,
    treated_col,
    cate_col,
    bins
    )

    It focuses on checking that lower valued bins have a lower
    uplift than higher valued ones, as well as for predictions too

    practically, we merge the above table with itself and then
    filter for right.bin_sort_id > left.bin_sort_id 
    (i.e if we set bins=10, then bin_sort_id=0 would represent the top bin
    whilst bin_sort_id=9 the bottom) to ensure we're comparing lower valued
    bins to a reference higher valued bin. We proceed by checking:
    * lift_diff_sign = sign(left.uplift - right.uplift)
    * pred_diff_sign = sign(left.pred_uplift - right.pred_uplift)
    * product = lift_diff_sign * pred_diff_sign

    Taking the mean of this product gives the kendall_rank_correlation
    which will be between -1 and 1. 
    """

    assert 'bin_sort_id' in df
    kendall_rank_corr_df = (
    df[['bin_sort_id', lift_col, pred_lift_col]].
        rename(columns={
        lift_col : 'lift',
        pred_lift_col : 'pred_cate'
        })
    ).copy()

    columne_renames = dict(
        bin_sort_id='comparison_bin_sort_id',
        lift='comparison_bin_lift',
        pred_cate='comparison_bin_pred_cate'        
    )
    kendall_rank_correlation_df = pd.merge(
        kendall_rank_corr_df.assign(merge_key=1),
        kendall_rank_corr_df.assign(merge_key=1).rename(columns=columne_renames),
        how='inner',
        left_on='merge_key',
        right_on='merge_key'
    ).query(
        'comparison_bin_sort_id>bin_sort_id'   
    ).assign(
        lift_diff=lambda x: x['lift'] - x['comparison_bin_lift'], # expect to be pos
        lift_diff_sign=lambda x: np.where(x['lift_diff'] >= 0, 1, -1),
        pred_diff=lambda x: x['pred_cate'] - x['comparison_bin_pred_cate'], # expect to be pos
        pred_diff_sign=lambda x: np.where(x['pred_diff'] >= 0, 1, -1),
        products=lambda x: x['lift_diff_sign'] * x['pred_diff_sign'],
        n=1,
    ).groupby(['bin_sort_id'], as_index=False)[['n','products']].sum()

    # note could have simply taken the mean(products) rather
    # than do it as I've done but above implements the idea
    # from looking at equation explicitely
    kendall_rank_correlation = (
        kendall_rank_correlation_df['products'].sum() / 
        kendall_rank_correlation_df['n'].sum()
    )

    return kendall_rank_correlation       

### getting metrics

In [12]:
train_lift_df = get_lift_effects_df(
    df=train_preds_df,
    outcome_col='y',
    treated_col='w',
    cate_col='cate',
    bins=10
).assign(
    split_type='train'
)

test_lift_df = get_lift_effects_df(
    df=test_preds_df,
    outcome_col='y',
    treated_col='w',
    cate_col='cate',
    bins=10
).assign(
    split_type='test'
)

In [13]:
train_lift_df

,n_tr,n_ct,y_tr,y_ct,cate_tr,cate_ct,lift,pred_cate,bin_sort_id,bin,global_lift,split_type
0,114,1386,0.517544,0.221501,19.947609,275.908085,0.296043,0.197237,0,0.12- 0.30,0.048386,train
1,200,1300,0.425000,0.158462,21.115254,138.749879,0.266538,0.106577,1,0.10- 0.12,0.048386,train
2,209,1291,0.306220,0.179706,19.452803,120.291875,0.126514,0.093163,2,0.09- 0.10,0.048386,train
3,207,1293,0.270531,0.179428,17.819866,111.055878,0.091104,0.085917,3,0.08- 0.09,0.048386,train
4,250,1250,0.228000,0.198400,19.848660,99.241419,0.029600,0.079393,4,0.08- 0.08,0.048386,train
5,245,1255,0.253061,0.217530,17.653582,90.549626,0.035531,0.072135,5,0.07- 0.08,0.048386,train
6,319,1181,0.216301,0.219306,19.820370,74.178319,-0.003005,0.062666,6,0.06- 0.07,0.048386,train
7,479,1021,0.244259,0.171401,23.584606,51.309439,0.072858,0.049929,7,0.04- 0.06,0.048386,train
8,661,839,0.186082,0.145411,25.195757,32.245901,0.040670,0.038294,8,0.03- 0.04,0.048386,train
9,686,814,0.160350,0.185504,16.210320,20.510374,-0.025154,0.024480,9,-0.02- 0.03,0.048386,train


### plotting - usual plot template

In [14]:
df_chart = pd.concat([
    train_lift_df[['bin_sort_id', 'split_type','pred_cate', 'global_lift']].rename(columns=dict(pred_cate='value')).assign(variable='pred'),
    train_lift_df[['bin_sort_id', 'split_type','lift', 'global_lift']].rename(columns=dict(lift='value')).assign(variable='obs'),

    test_lift_df[['bin_sort_id', 'split_type','pred_cate', 'global_lift']].rename(columns=dict(pred_cate='value')).assign(variable='pred'),
    test_lift_df[['bin_sort_id', 'split_type','lift', 'global_lift']].rename(columns=dict(lift='value')).assign(variable='obs'),

], axis=0) 


In [15]:
overall_chart = alt.Chart(df_chart).mark_rule(color='black', opacity=0.8, strokeDash=[2,1]).encode(
    y=alt.Y('min(global_lift)', title='global lift'),    
)

chart = alt.Chart(df_chart).mark_bar().encode(    
    y=alt.Y('value', title=None),
    color=alt.Color('variable', sort=['pred', 'obs']),
    x=alt.X('variable', sort=['pred', 'obs'], axis=alt.Axis(labels=False, ticks=False), title=None),
    text=alt.Text('value', format='0.2f'),
    tooltip=[
        alt.Tooltip('variable'),        
        alt.Tooltip('value', format='0.3f'),
    ]
)
chart = chart + chart.mark_text(align='center', dy=-10, )
chart = chart + overall_chart
chart = chart.properties(
    width=50, height=125
).facet(
    column=alt.Column('bin_sort_id', title=None),
    row=alt.Row('split_type', sort=['train', 'test'], title=None)
)

chart

alt.FacetChart(...)

### plotting - scatter plot template

In [16]:
alt.Chart(pd.concat([train_lift_df,test_lift_df], axis=0)).mark_point(filled=True).encode(
  x='pred_cate',
  y='lift',
  color='split_type',
  facet=alt.Facet('split_type',columns=2, sort=['train', 'test'])
).properties(
  width=250, height=250
)

alt.Chart(...)

### what about a random model?

In [17]:
train_lift_df_2 = []
for cate_col in ['cate'] + [f'RANDOM_{idx}' for idx in range(1, 11)]:    
    # finally using master report function
    train_lift_df = get_lift_effects_df(
        df=train_preds_df,
        outcome_col='y',
        treated_col='w',
        cate_col=cate_col,
        bins=10
    ).assign(
        split_type='train',
        cate_col=cate_col
    )

    train_lift_df_2.append(train_lift_df)

train_lift_df_2 = pd.concat(train_lift_df_2, axis=0).reset_index(drop=True)    

In [18]:
train_lift_df_2.head()

,n_tr,n_ct,y_tr,y_ct,cate_tr,cate_ct,lift,pred_cate,bin_sort_id,bin,global_lift,split_type,cate_col
0,114,1386,0.517544,0.221501,19.947609,275.908085,0.296043,0.197237,0,0.12- 0.30,0.048386,train,cate
1,200,1300,0.425000,0.158462,21.115254,138.749879,0.266538,0.106577,1,0.10- 0.12,0.048386,train,cate
2,209,1291,0.306220,0.179706,19.452803,120.291875,0.126514,0.093163,2,0.09- 0.10,0.048386,train,cate
3,207,1293,0.270531,0.179428,17.819866,111.055878,0.091104,0.085917,3,0.08- 0.09,0.048386,train,cate
4,250,1250,0.228000,0.198400,19.848660,99.241419,0.029600,0.079393,4,0.08- 0.08,0.048386,train,cate


In [19]:
overall_chart = alt.Chart(train_lift_df_2).mark_rule(color='black', size=2, opacity=0.8, strokeDash=[2,4]).encode(
    y=alt.Y('min(global_lift)', title='global lift'),    
)

chart = alt.Chart(train_lift_df_2).mark_line(size=1).encode(
    y='lift',
    x='bin_sort_id:O',
    color='cate_col'
).properties(
    width=550, height=250
)

chart = chart + overall_chart
chart

alt.LayerChart(...)

* reference random model is given by dotted line
  * is equal to the global lift across all bins (i.e. on avg. in any bin there is a mix of pos/neg
treatment effects on customers)

### what about correlation?

In [20]:
dict(
    train=f'{kendall_rank_correlation(df=train_lift_df) : 0.3f}',
    test=f'{kendall_rank_correlation(df=test_lift_df) : 0.3f}'
)

{'train': ' 0.156', 'test': ' 0.644'}

In [21]:
auc = dict(
    train=((train_lift_df['lift'] - train_lift_df['global_lift']).abs()*1/10).sum(),
    test=((test_lift_df['lift'] - test_lift_df['global_lift']).abs()*1/10).sum(),    
)
auc

{'train': 0.012159737191972093, 'test': 0.02737852645813278}

In [22]:
weighted_auc = dict(
    train=((train_lift_df['lift'] - train_lift_df['global_lift']).abs()*1/10).sum() * kendall_rank_correlation(df=train_lift_df),
    test=((test_lift_df['lift'] - test_lift_df['global_lift']).abs()*1/10).sum() * kendall_rank_correlation(df=test_lift_df),    
)
weighted_auc

{'train': 0.00189151467430677, 'test': 0.017643939273018903}

## Cumulative Lift (100 bins)

### helper function

In [23]:
def generate_binned_report(
    df: pd.DataFrame,
    outcome_col: str,
    treated_col: str,
    cate_col: str,
    bins: int,
):
    """
    Each bin will have n_treated customers (n_tr) with y_treated values (y_tr).
    In addition we can also estimate the cumsum versions of these cumsum_n_tr, cumsum_y_tr.
    Similarly we will have this for controls (n_ct, y_ct, cumsum_n_ct, cumsum_y_ct).
    We also will have predited cate for treated and control (cate_tr, cate_ct, cumsum_cate_tr, cumsum_cate_ct).

    outputs a data_frame with n_row equal to the number of bins specified with fields:
        output 1 pd.DataFrame:
            id_vars:
                bin_sort_id
                bin
            bin_level_info:
                n_tr: # of treated customers assigned to a bin
                n_ct: # of control customers assigned to a bin
                y_tr: avg. outcome value for treated assigned to a bin
                y_ct: avg. outcome value for control assigned to a bin
                lift: (y_tr -  y_ct) assigned to a bin
                cate_tr: predicted conditional avg. treatment effect for treated group assigned to a bin
                cate_ct: predicted conditional avg. treatment effect for control group assigned to a bin   
                pred_cate: predicted conditional avg. treatment effect (treated + control group) assigned to a bin
            cumsum_level_info (if not specified desc below is the same as above but cumsum version of it):
                cumsum_n_tr: 
                cumsum_n_ct: 
                cumsum_y_tr: 
                cumsum_y_ct:
                cumsum_lift:                
                cumsum_cate_tr:
                cumsum_cate_ct:
                cumsum_pred_cate:
                cumsum_n: cumsum_n_tr + cumsum_n_ct
                depth: this represents cumsum % of base contacted, which is x['cumsum_n'] / x['cumsum_n'].max()
                gains: x['cumsum_lift'] * x['depth']
                gains_RANDOM_MODEL: x['global_lift'] * x['depth'] i.e. a random model will be a linear line from 0 to
                    the global_lift i.e. a random model will not discrimate high/low true cate customers but will
                    always have a blend of them, and this blend will eventually reach the overall.
                normalised_gains: (x['cumsum_lift'] - x['global_lift']) * x['depth']
                normalised_gains_RANDOM_MODEL: will theoretically be 0 always:
                global_lift: 
                    last record of cumsum_lift i.e. x.iloc[-1]['cumsum_lift'] 
                    or
                        y_global_tr: (x['y_tr'] * x['n_tr']).sum() / x['n_tr'].sum()
                        y_global_ct: (x['y_ct'] * x['n_ct']).sum() / x['n_ct'].sum()
                        global_lift: (y_global_tr - y_global_ct)
                lift_auc: (x['lift'] - x['global_lift']).abs() * 1/n (i.e. dy * dx to get area above random lift)
                cumsum_lift_auc: (x['cumsum_lift']- x['global_lift']) * 1/n_bins
                gains_auc: x['gains'] * 1/n
                normalised_gains_auc: x['normalised_gains'] * 1/n
                gains_auc_RANDOM_MODEL: x['gains_RANDOM_MODEL'] * 1/n                                                               
        output 2 auc summary info:
            lift_auc:
                desc: area between the lift curve and global_lift summed all up
                calc: bin_rep_df['lift_auc'].sum()
            cumsum_lift_auc:
                desc: area between the cumulative lift curve and global lift summed all up
                calc: bin_rep_df['cumsum_lift_auc'].sum()
            gains_auc:
                desc: area between cumsum gains curve
                calc: bin_rep_df['gains_auc'].sum()
            gains_auc_RANDOM_MODEL:
                desc: area between cumsum random model curve
                calc: bin_rep_df['gains_auc_RANDOM_MODEL'].sum()                
            gains_auc_above_RANDOM_MODEL:
                desc: area between cumsum gains curve vs. random cumsum gains curve
                calc: bin_rep_df['gains_auc'].sum()-bin_rep_df['gains_auc_RANDOM_MODEL'].sum()            
            normalised_gains_auc:
                desc: area between cumsum normalised gains curve. Nice interpretation of 0 being random model
                calc: bin_rep_df['normalised_gains_auc'].sum()


    """
    bin_rep_df = get_lift_effects_df(
      df,
      outcome_col,
      treated_col,
      cate_col,
      bins
    )

    n = bin_rep_df.shape[0]

    bin_rep_df = bin_rep_df.assign(
        # y_tr & y_ct are in avg form need to remove this
        # they are avg for a bin
        y_tr=lambda x: x['y_tr'] * x['n_tr'],
        y_ct=lambda x: x['y_ct'] * x['n_ct'], 
        lift=lambda x: (x['y_tr']/x['n_tr']) - (x['y_ct']/x['n_ct']), # was already there but re-calculating
        # cate_tr and cate_ct wasn't in avg form so can do the below
        pred_cate = lambda x: (x['cate_tr'] + x['cate_ct']) / (x['n_tr']+ x['n_ct'])
    ).assign(
        cumsum_n_tr=lambda x: x['n_tr'].cumsum(),
        cumsum_n_ct=lambda x: x['n_ct'].cumsum(),
        cumsum_y_tr=lambda x: x['y_tr'].cumsum(),
        cumsum_y_ct=lambda x: x['y_ct'].cumsum(),
        cumsum_cate_tr=lambda x: x['cate_tr'].cumsum(),
        cumsum_cate_ct=lambda x: x['cate_ct'].cumsum(),
        cumsum_lift=lambda x: (x['cumsum_y_tr']/x['cumsum_n_tr']) - (x['cumsum_y_ct']/x['cumsum_n_ct']),
        cumsum_pred_cate=lambda x: (x['cumsum_cate_tr'] + x['cumsum_cate_ct']) / (x['cumsum_n_tr'] + x['cumsum_n_ct']),
        cumsum_n=lambda x: x['cumsum_n_tr'] +  x['cumsum_n_ct'],
        depth=lambda x: x['cumsum_n'] / x['cumsum_n'].max(),
        gains=lambda x: x['cumsum_lift'] * x['depth'],
        global_lift=lambda x: x.iloc[-1]['cumsum_lift'],
        normalised_gains=lambda x: (x['cumsum_lift'] - x['global_lift']) * x['depth']
    ).assign(
        # auc for lift first based on global lift area summed
        # note that we value a negative value as well for this! i.e. it's still a positive thing
        # to be very negative but this is for the bottom bins more than the top bins!
        lift_auc=lambda x: (x['lift'] - x['global_lift']).abs() * 1/n,
        cumsum_lift_auc=lambda x: (x['cumsum_lift'] - x['global_lift']) * 1/n,
        # add random model for gains
        gains_RANDOM_MODEL=lambda x: x['global_lift'] * x['depth'],
        # get auc for gains
        gains_auc=lambda x: x['gains'] * 1/n,
        # get auc for gains for random moodel
        gains_auc_RANDOM_MODEL=lambda x: x['gains_RANDOM_MODEL'] * 1/n,
        # add random model for normalised gains
        normalised_gains_RANDOM_MODEL=0,
        # get auc for normalised gains
        normalised_gains_auc=lambda x: x['normalised_gains'] * 1/n,
        # 
    )

    fields_keeping = [
        'bin_sort_id',
        'bin',
        'y_tr',
        'y_ct',
        'n_tr', 
        'n_ct',
        'cate_tr', 
        'cate_ct',
        'lift',
        'pred_cate',
        'cumsum_y_tr', 
        'cumsum_y_ct', 
        'cumsum_n_tr', 
        'cumsum_n_ct', 
        'cumsum_cate_tr',
        'cumsum_cate_ct',
        'cumsum_lift', 
        'cumsum_pred_cate',
        'cumsum_n', 
        'depth',
        'gains', 
        'gains_RANDOM_MODEL', 
        'normalised_gains', 
        'normalised_gains_RANDOM_MODEL', 
        'global_lift',         
        'lift_auc', # ((lift - global_lift) * 1/n_bins).sum()        
        'cumsum_lift_auc', # ((cumsum_lift - global_lift) * 1/n_bins).sum()
        'gains_auc',
        'normalised_gains_auc', 
        'gains_auc_RANDOM_MODEL'
    ]

    bin_rep_df = bin_rep_df[fields_keeping]

    auc_summary = dict(
        lift_auc=bin_rep_df['lift_auc'].sum(),
        cumsum_lift_auc=bin_rep_df['cumsum_lift_auc'].sum(),
        gains_auc=bin_rep_df['gains_auc'].sum(),
        gains_auc_RANDOM_MODEL=bin_rep_df['gains_auc_RANDOM_MODEL'].sum(),
        gains_auc_above_RANDOM_MODEL=bin_rep_df['gains_auc'].sum()-bin_rep_df['gains_auc_RANDOM_MODEL'].sum(),
        normalised_gains_auc=bin_rep_df['normalised_gains_auc'].sum()
    )

    
    return bin_rep_df, auc_summary

### getting metrics

In [24]:
train_rep_df, train_auc_summary = generate_binned_report(
    df=train_preds_df,
    outcome_col='y',
    treated_col='w',
    cate_col='cate',
    bins=100,
)

In [25]:
all_reports_df = []
all_area_under_curve_summaries = {}
for cate_col in ['cate'] + [f'RANDOM_{idx}' for idx in range(1, 11)]:    

    train_rep_df, train_auc_summary = generate_binned_report(
        df=train_preds_df,
        outcome_col='y',
        treated_col='w',
        cate_col=cate_col,
        bins=100,
    )
    
    train_rep_df = train_rep_df.assign(
        split_type='train',
        cate_col=cate_col
    )


    test_rep_df, test_auc_summary = generate_binned_report(
        df=test_preds_df,
        outcome_col='y',
        treated_col='w',
        cate_col=cate_col,
        bins=100,
    )
    
    test_rep_df = test_rep_df.assign(
        split_type='test',
        cate_col=cate_col
    )

    all_reports_df.append(train_rep_df)
    all_reports_df.append(test_rep_df)

    all_area_under_curve_summaries[cate_col] = dict(
        train=train_auc_summary,
        test=test_auc_summary
    )

all_reports_df = pd.concat(all_reports_df, axis=0).reset_index(drop=True)
all_reports_df.head()

,bin_sort_id,bin,y_tr,y_ct,n_tr,n_ct,cate_tr,cate_ct,lift,pred_cate,...,normalised_gains,normalised_gains_RANDOM_MODEL,global_lift,lift_auc,cumsum_lift_auc,gains_auc,normalised_gains_auc,gains_auc_RANDOM_MODEL,split_type,cate_col
0,0,0.27- 0.30,7.0,34.0,7,143,1.942180,40.104167,0.762238,0.280309,...,0.007139,0,0.048386,0.007139,0.007139,0.000076,0.000071,0.000005,train,cate
1,1,0.26- 0.27,5.0,42.0,6,144,1.586116,38.000135,0.541667,0.263908,...,0.012198,0,0.048386,0.004933,0.006099,0.000132,0.000122,0.000010,train,cate
2,2,0.24- 0.26,3.0,53.0,7,143,1.771519,35.759254,0.057942,0.250205,...,0.012048,0,0.048386,0.000096,0.004016,0.000135,0.000120,0.000015,train,cate
3,3,0.22- 0.24,3.0,46.0,3,147,0.705156,34.410029,0.687075,0.234101,...,0.017237,0,0.048386,0.006387,0.004309,0.000192,0.000172,0.000019,train,cate
4,4,0.19- 0.22,5.0,39.0,10,140,2.081600,29.230029,0.221429,0.208744,...,0.017506,0,0.048386,0.001730,0.003501,0.000199,0.000175,0.000024,train,cate


### plotting results

In [26]:
mask = all_reports_df['cate_col'] == 'cate'
mask_2 = all_reports_df['cate_col'] != 'cate'


df_chart = pd.concat([
    all_reports_df[mask][['bin_sort_id','split_type' ,'depth', 'cumsum_lift']].rename(columns=dict(cumsum_lift='cumulative_lift')).assign(type='lift'),
    all_reports_df[mask][['bin_sort_id','split_type' ,'depth', 'cumsum_pred_cate']].rename(columns=dict(cumsum_pred_cate='cumulative_lift')).assign(type='predicted_lift'),    
    all_reports_df[mask][['bin_sort_id', 'split_type','depth', 'global_lift']].rename(columns=dict(global_lift='cumulative_lift')).assign(type='global_lift'),    
    all_reports_df[mask_2][['bin_sort_id', 'split_type','cate_col','depth', 'cumsum_lift']].rename(columns=dict(cumsum_lift='cumulative_lift', cate_col='type'))
], axis=0)

df_chart.head()

,bin_sort_id,split_type,depth,cumulative_lift,type
0,0,train,0.01,0.762238,lift
1,1,train,0.02,0.658269,lift
2,2,train,0.03,0.450000,lift
3,3,train,0.04,0.479316,lift
4,4,train,0.05,0.398504,lift


In [27]:
df_chart.groupby(['type']).size()

type
RANDOM_1          200
RANDOM_10         200
RANDOM_2          200
RANDOM_3          200
RANDOM_4          200
RANDOM_5          200
RANDOM_6          200
RANDOM_7          200
RANDOM_8          200
RANDOM_9          200
global_lift       200
lift              200
predicted_lift    200
dtype: int64

In [28]:
list(color_rules.values())

NameError: name 'color_rules' is not defined

In [ ]:
color_rules = {
    'predicted_lift' : '#fd8d3c',    
    'lift' : '#3182bd',
    'RANDOM_1' : '#252525',
    'RANDOM_2' : '#525252',
    'RANDOM_3' : '#737373',
    'RANDOM_4' : '#969696',
    'RANDOM_5' : '#bdbdbd',
    #'RANDOM_6' : '#252525',
    #'RANDOM_7' : '#525252',
    #'RANDOM_8' : '#737373',
    #'RANDOM_9' : '#969696',
    #'RANDOM_10' : '#bdbdbd',
    'global_lift' : 'darkgreen',
} 


mask = df_chart['type'].isin(color_rules.keys())
alt.Chart(df_chart[mask]).mark_line(point=alt.OverlayMarkDef(filled=True, size=20)).encode(
    x='depth',
    y='cumulative_lift',
    color=alt.Color(
        'type',
        scale=alt.Scale(
            domain=list(color_rules.keys()),
            range=list(color_rules.values())
        )
    ),
    tooltip=[
        "type",
        alt.Tooltip("cumulative_lift", format="0.2f")
    ]
).properties(
    width=350
).facet(
    column=alt.Column('split_type')
)

alt.FacetChart(...)

In [ ]:
for cate_col in ['cate'] + [f'RANDOM_{idx}' for idx in range(1, 11)]:
    df_case = all_reports_df.query('split_type=="train"').query('cate_col==@cate_col')
    corr_train=kendall_rank_correlation(df=df_case, lift_col='cumsum_lift', pred_lift_col='cumsum_pred_cate')
    kendall_rank_corr = dict(
        corr=f'{corr_train : 0.3f}',
        cate_col=cate_col
    )
    print(kendall_rank_corr)

{'corr': ' 0.980', 'cate_col': 'cate'}
{'corr': '-0.423', 'cate_col': 'RANDOM_1'}
{'corr': '-0.015', 'cate_col': 'RANDOM_2'}
{'corr': ' 0.054', 'cate_col': 'RANDOM_3'}
{'corr': '-0.623', 'cate_col': 'RANDOM_4'}
{'corr': ' 0.753', 'cate_col': 'RANDOM_5'}
{'corr': ' 0.080', 'cate_col': 'RANDOM_6'}
{'corr': '-0.086', 'cate_col': 'RANDOM_7'}
{'corr': ' 0.386', 'cate_col': 'RANDOM_8'}
{'corr': '-0.793', 'cate_col': 'RANDOM_9'}
{'corr': ' 0.152', 'cate_col': 'RANDOM_10'}


In [ ]:
# or the actual bin's observed lift vs. pred lift/cate....however, as the above is based on cumulative lift, I believe it makes sense to
# use a cumulative evaluation metric here
for cate_col in ['cate'] + [f'RANDOM_{idx}' for idx in range(1, 11)]:
    df_case = all_reports_df.query('split_type=="train"').query('cate_col==@cate_col')
    corr_train=kendall_rank_correlation(df=df_case, lift_col='lift', pred_lift_col='pred_cate')
    kendall_rank_corr = dict(
        corr=f'{corr_train : 0.3f}',
        cate_col=cate_col
    )
    print(kendall_rank_corr)

{'corr': ' 0.433', 'cate_col': 'cate'}
{'corr': '-0.084', 'cate_col': 'RANDOM_1'}
{'corr': ' 0.009', 'cate_col': 'RANDOM_2'}
{'corr': '-0.140', 'cate_col': 'RANDOM_3'}
{'corr': '-0.012', 'cate_col': 'RANDOM_4'}
{'corr': ' 0.129', 'cate_col': 'RANDOM_5'}
{'corr': '-0.009', 'cate_col': 'RANDOM_6'}
{'corr': ' 0.059', 'cate_col': 'RANDOM_7'}
{'corr': ' 0.015', 'cate_col': 'RANDOM_8'}
{'corr': '-0.088', 'cate_col': 'RANDOM_9'}
{'corr': ' 0.017', 'cate_col': 'RANDOM_10'}


In [ ]:
all_area_under_curve_summaries['cate']['train']['cumsum_lift_auc']

0.12845620804007968

In [ ]:
all_area_under_curve_summaries['RANDOM_1']['train']['cumsum_lift_auc']

-0.009731856852989527

In [ ]:
all_area_under_curve_summaries['cate']['train']['lift_auc'] # this normally is found for bins=10

0.11830571204387828

In [ ]:
all_area_under_curve_summaries['RANDOM_1']['train']['lift_auc']

0.05777590571596558

## Cumulative Depth (100 bins)

### getting metrics

In [ ]:
theoretical_random = (
    all_reports_df.
        query('cate_col=="cate"')
        [['bin_sort_id', 'depth', 'split_type', 'gains_RANDOM_MODEL']].
        assign(cate_col='theoretical-random').
        rename(columns=dict(gains_RANDOM_MODEL='gains'))
)

theoretical_random.shape

(200, 5)

In [ ]:
gains_chart =  pd.concat([
    all_reports_df[['bin_sort_id', 'depth', 'cate_col', 'split_type','gains']],
    theoretical_random
], axis=0)

In [ ]:

my_chart = alt.Chart(gains_chart).mark_line().encode(
    y=alt.Y('gains', scale=alt.Scale(zero=False)),
    x=alt.X('depth'),
    color='cate_col',
    tooltip=[
        'depth',
        'gains',
    ],
    facet=alt.Facet('split_type', title=None, columns=2, sort=['train', 'test'])
).properties(
    title='N/A'
)

my_chart





alt.Chart(...)

### comment

you can define an area under the cumulative gains chart/curve between the cate/model curve and theoretical random curve

In [ ]:
dict(
    gains_auc=all_area_under_curve_summaries['cate']['train']['gains_auc'],
    gains_auc_RANDOM_MODEL=all_area_under_curve_summaries['cate']['train']['gains_auc_RANDOM_MODEL'],
    gains_auc_above_RANDOM_MODEL=all_area_under_curve_summaries['cate']['train']['gains_auc']-all_area_under_curve_summaries['cate']['train']['gains_auc_RANDOM_MODEL']
)


{'gains_auc': 0.05947784815396295,
 'gains_auc_RANDOM_MODEL': 0.024435093166909474,
 'gains_auc_above_RANDOM_MODEL': 0.035042754987053476}

In [ ]:
all_reports_df.assign(
    gains_auc_above_RANDOM_MODEL=lambda x:x['gains_auc'] - x['gains_auc_RANDOM_MODEL']
).groupby(['cate_col','split_type'], as_index=False)['gains_auc_above_RANDOM_MODEL'].sum().pivot_table(
    index='cate_col',
    columns='split_type',
    values='gains_auc_above_RANDOM_MODEL'
).sort_values(
    ['train'], ascending=False
)

split_type,test,train
cate_col,,
cate,0.008341,0.035043
RANDOM_5,-0.002352,0.004523
RANDOM_10,0.001499,0.001062
RANDOM_7,0.002358,0.000429
RANDOM_2,-0.000832,0.000220
RANDOM_4,-0.001271,-0.000323
RANDOM_6,-0.000768,-0.000437
RANDOM_8,0.001679,-0.000576
RANDOM_9,0.001472,-0.002004


## Normalised Cumulative Depth (100 bins)

In [ ]:
theoretical_random = (
    all_reports_df.
        query('cate_col=="cate"')
        [['bin_sort_id', 'depth', 'split_type', 'normalised_gains_RANDOM_MODEL']].
        assign(cate_col='theoretical-random').
        rename(columns=dict(normalised_gains_RANDOM_MODEL='normalised_gains'))
)

all_reports_df.shape


(2200, 32)

In [ ]:
gains_chart =  pd.concat([
    all_reports_df[['bin_sort_id', 'depth', 'cate_col', 'split_type','normalised_gains']],
    theoretical_random
], axis=0)

In [ ]:

my_chart = alt.Chart(gains_chart).mark_line().encode(
    y=alt.Y('normalised_gains', scale=alt.Scale(zero=False)),
    x=alt.X('depth'),
    color='cate_col',
    tooltip=[
        'depth',
        'normalised_gains',
    ],
    facet=alt.Facet('split_type', title=None, columns=2, sort=['TrainPreds', 'TunePreds', 'TestPeds'])
).properties(
    title='N/A'
)

my_chart

alt.Chart(...)

#### comment

you can define the area under the normalised gains chart which above will be from the cate/model curve to the x-axis. An AUC that's greater than 0 is a sign that the model is performant

In [ ]:
dict(
    gains_auc_above_RANDOM_MODEL=all_area_under_curve_summaries['cate']['train']['gains_auc']-all_area_under_curve_summaries['cate']['train']['gains_auc_RANDOM_MODEL'],
    normalised_gains_auc=all_area_under_curve_summaries['cate']['train']['normalised_gains_auc']
)

{'gains_auc_above_RANDOM_MODEL': 0.035042754987053476,
 'normalised_gains_auc': 0.03504275498705348}

In [29]:
all_reports_df.groupby(['cate_col','split_type'], as_index=False)['normalised_gains_auc'].sum().pivot_table(
    index='cate_col',
    columns='split_type',
    values='normalised_gains_auc'
).sort_values(
    ['train'], ascending=False
)

split_type,test,train
cate_col,,
cate,0.008341,0.035043
RANDOM_5,-0.002352,0.004523
RANDOM_10,0.001499,0.001062
RANDOM_7,0.002358,0.000429
RANDOM_2,-0.000832,0.000220
RANDOM_4,-0.001271,-0.000323
RANDOM_6,-0.000768,-0.000437
RANDOM_8,0.001679,-0.000576
RANDOM_9,0.001472,-0.002004
